## Notebook for computing consistency for model set pairs across datasets and similarity metrics
This notebook computes for s.

In [ ]:
from itertools import combinations, product

import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr

from constants import (
    exclude_models,
    exclude_models_w_mae,
    cat_name_mapping,
    model_config_file,
    model_categories,
    sim_metric_name_mapping,
    model_cat_mapping,
    BASE_PATH_PROJECT,
    BASE_PATH_RESULTS,
    ds_list_sim_file
)
from helper import (
    load_model_configs_and_allowed_models,
    load_similarity_matrices,
    pp_storing_path,
    load_all_datasetnames_n_info
)

#### Global variables

In [ ]:
base_path_similarity_matrices = BASE_PATH_PROJECT / 'model_similarities'
sim_metrics = [
    'cka_kernel_rbf_unbiased_sigma_0.4',
    'cka_kernel_linear_unbiased',
    'rsa_method_correlation_corr_method_spearman'
]
sim_metrics_mapped = [sim_metric_name_mapping[k] for k in sim_metrics]

ds_list, _ = load_all_datasetnames_n_info(ds_list_sim_file, verbose=False)

# Define model set pair:
# - A model set pair is a tuple of two lists, where each list specifies model selection criteria
# - Each list can contain either:
#   a) Model categories/subcategories (e.g., ('objective', 'Image-Text'))
#   b) Specific model names with their config column (e.g., ('index', 'mae-vit-base-p16'))
#
# Examples:
# 1. Comparing model categories:
#    ([('objective', 'Image-Text')],  # First set: only image-text models
#     [('objective', 'Image-Text'),   # Second set: union of image-text and supervised models
#      ('objective', 'Supervised')])
#
mode_set_pair_1 = ([('+', 'objective', 'Image-Text')],                # First set: all image-text models
                   [('+', 'objective', 'Image-Text'),                 # Second set: image-text models plus
                    ('+', 'index', 'mae-vit-base-p16'),              # specific MAE models
                    ('+', 'index', 'mae-vit-huge-p14'),
                    ('+', 'index', 'mae-vit-large-p16')])

mode_set_pair_2 = ([('+', 'objective', 'Image-Text')], 
                   [('+', 'objective', 'Image-Text'), 
                    ('+', 'objective', 'Supervised')])

mode_set_pair_3 = ([('+', 'objective', 'Self-Supervised'), 
                    ('-', 'index', 'mae-vit-base-p16'),    
                    ('-', 'index', 'mae-vit-huge-p14'),
                    ('-', 'index', 'mae-vit-large-p16')],  
                   [('+', 'objective', 'Self-Supervised'), 
                    ('-', 'index', 'mae-vit-base-p16'),
                    ('-', 'index', 'mae-vit-huge-p14'),
                    ('-', 'index', 'mae-vit-large-p16')])

curr_mode_set_pair = mode_set_pair_2

#### Load model configurations and similarity matrices

In [ ]:
model_configs, allowed_models = load_model_configs_and_allowed_models(
    path=model_config_file,
    exclude_models=[],
    exclude_alignment=True,
)

In [ ]:
def get_model_sets(model_configs, mode_set_pair):
    """
    Extract model IDs (indices) for each model set based on specified criteria.
    
    Parameters:
    -----------
    model_configs : pd.DataFrame
        DataFrame containing model configurations with model characteristics
    mode_set_pair : tuple of lists
        Tuple containing two lists, each specifying selection criteria for a model set
        Each list can contain tuples of (column_name, value) for filtering
    
    Returns:
    --------
    tuple
        Tuple of two sets containing model IDs (indices) for each model set
    """
    model_sets = []
    
    # Process each model set criteria
    for criteria_list in mode_set_pair:
        # Start with all model indices
        current_set = set()
        
        # Apply each criterion
        for update_type, column, value in criteria_list:
            # Filter models that match the current criterion
            if column == 'index':
                if value not in model_configs.index:
                    raise ValueError(f"Value not in `model_configs.index`. Passed (column, value) pair invalid!")
                matching_models = set([value])
            else:
                matching_models = set(model_configs[model_configs[column] == value].index)
            if update_type == '+':
                current_set.update(matching_models)
            elif update_type == '-':
                current_set -= matching_models
            else:
                raise ValueError(f'Unknown update type {update_type}. Allowed values ["+", "-"]')
                
            
        model_sets.append(current_set)
    if any([len(m_set)<=0 for m_set in model_sets]):
        raise ValuesError('For current model set pair configuration no models have been found!')
    
    return tuple(model_sets)


def get_model_combinations(model_sets):
    """
    Generate all possible combinations of models between two model sets.
    """
    set1, set2 = model_sets
    combinations = list(set([tuple(sorted([model1, model2])) for model1 in set1 for model2 in set2 if model1!=model2]))
    return combinations

In [ ]:
model_sets = get_model_sets(model_configs, curr_mode_set_pair)
allowed_model_combinations = get_model_combinations(model_sets)

In [ ]:
len(allowed_model_combinations)

In [ ]:
sim_mats = load_similarity_matrices(
    path=base_path_similarity_matrices,
    ds_list=ds_list,
    sim_metrics=sim_metrics,
    allowed_models=allowed_models,
)
sim_mats = {sim_metric_name_mapping[k]: v for k, v in sim_mats.items()}
sim_mats.keys()

#### Helper functions

In [ ]:
def get_sim_data(category, sim_metric):
    cat_groups = model_configs.reset_index(names=['mid']).groupby(category)['mid'].unique()
    cat_groups.index = [cat_name_mapping[x] for x in cat_groups.index]
    sim_metric_mats = sim_mats[sim_metric]
    return cat_groups, sim_metric_mats


def process_sim_mat(ds, sim_mat):
    indx_i, indx_j = np.triu_indices(n=len(sim_mat), k=1)
    flat_sim_mat = pd.DataFrame(
        {'Similarity value': sim_mat.values[indx_i, indx_j],
         'Model 1': sim_mat.index.values[indx_i],
         'Model 2': sim_mat.columns.values[indx_j]
         })
    flat_sim_mat['DS'] = ds
    return flat_sim_mat


def r_coeff(df, ds1, ds2, corr_type):
    x = df[df['DS'] == ds1]['Similarity value']
    y = df[df['DS'] == ds2]['Similarity value']
    if 'pearsonr' == corr_type:
        corr, _ = pearsonr(x, y)
    elif 'spearmanr' == corr_type:
        corr, _ = spearmanr(x, y)
    else:
        raise ValueError(f'Unknown {corr_type=}! Need to select pearsonr or spearmanr r_corr in corr_type.')
    return corr


def get_r_coeff_df(sim_metric_mats, corr_type, allowed_model_combinations):
    pp_mats = []
    for ds, sim_mat in sim_metric_mats.items():
        pp_mats.append(process_sim_mat(ds, sim_mat))
    pp_mats = pd.concat(pp_mats).reset_index(drop=True)
    
    pp_mats['Model pair'] = pp_mats[['Model 1', 'Model 2']].apply(lambda x: tuple(sorted([x['Model 1'], x['Model 2']])), axis=1)
    
    pp_mats = pp_mats[pp_mats['Model pair'].isin(allowed_model_combinations)].reset_index(drop=True)

    combs = list(combinations(list(pp_mats['DS'].unique()), 2))
    r_coeffs = []
    for ds1, ds2 in combs:
        subset_ds = pp_mats[pp_mats['DS'].isin([ds1, ds2])]
        r_coeffs.append({
            'ds1': ds1,
            'ds2': ds2,
            'r coeff': r_coeff(subset_ds, ds1, ds2, corr_type),
            })
    df_r_coeffs = pd.DataFrame(r_coeffs)
    return df_r_coeffs
    

### Aggregating r-coefficients for model set pairs across datasets and similarity metrics

In [ ]:
for corr_type, sim_metric in product(['pearsonr', 'spearmanr'], list(sim_mats.keys())):
    sim_metric_mats = sim_mats[sim_metric]
    df_r_coeffs = get_r_coeff_df(sim_metric_mats, corr_type, allowed_model_combinations)

    print(corr_type, sim_metric)
    print(df_r_coeffs['r coeff'].describe())
    print()